Installing the Julep Client



In [12]:
# !pip install julep -U --quiet


In [13]:
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

### Creating Julep client with the api key

In [ ]:
from julep import Client
import os

JULEP_API_KEY = os.environ["JULEP_API_KEY"]

# Create a Julep client
client = Client(api_key='secret API key', environment="production")

## LEts Create an "agent"

In [15]:
# Defining the agent
name = "RVC"
about = "A travel assistant that helps plan the perfect trip."

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

In [ ]:
import yaml
task_def = yaml.safe_load("""
name: Foodie Tour Generator
description: Creates a 1-day foodie tour using weather, local dishes, and top restaurants

input_schema:
  type: object
  properties:
    location:
      type: string
      description: The city to generate the food tour for

tools:
  - name: weather_check
    type: integration
    integration:
      provider: weather
      method: get
      setup:
        openweathermap_api_key: "Secret Api key"

  - name: dish_finder
    type: integration
    integration:
      provider: brave
      method: search
      setup:
        brave_api_key: "Secret Api key"

  - name: restaurant_finder
    type: integration
    integration:
      provider: brave
      method: search
      setup:
        brave_api_key: "Secret API key"

main:
  # Step 1: Get weather data
  - tool: weather_check
    arguments:
      location: $ steps[0].input.location
    as: weather_data

  # Step 2: Find local dishes
  - tool: dish_finder
    arguments:
      query: $ "3 iconic local dishes of " + steps[0].input.location              

  # Step 3: Find restaurants
  - tool: restaurant_finder
    arguments:
      query: $ "best restaurants serving [popular dishes] in " + steps[0].input.location
    as: restaurants


  - prompt:
    - role: system
      content: |
        You are a food-loving travel writer who creates delightful one-day food tours.
        Combine weather conditions, local specialties, and restaurant recommendations
        into a fun, engaging narrative that reads like a food adventure story.
    - role: user
      content: |
       
        Create a 1-day food tour for {steps[0].input.location} with:
          - Current weather: {steps[0].output}
          - Must-try dishes: {steps[1].output}
          - Recommended restaurants: {steps[2].output}
          
          Structure:
          1. Weather-appropriate introduction
          2. Breakfast spot with dish recommendation
          3. Lunch experience with local specialty
          4. Dinner venue matching the weather mood
          5. Closing food adventure tip or some nearby place to visit after having food .
          
          Make it vivid, sensory, intriging, and full of personality!
                                 
          Special instructions:
           "If rainy/chilly": "Suggest cozy indoor spots with comfort food",
            "If sunny/warm": "Include outdoor dining with scenic views",
            "Always": "Include 1 hidden gem locals love" 
            and write it all using emoji
             
""")

In [17]:
print(task_def)

{'name': 'Foodie Tour Generator', 'description': 'Creates a 1-day foodie tour using weather, local dishes, and top restaurants', 'input_schema': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city to generate the food tour for'}}}, 'tools': [{'name': 'weather_check', 'type': 'integration', 'integration': {'provider': 'weather', 'method': 'get', 'setup': {'openweathermap_api_key': 'aebf67ced614806bff70a47aab381b9a'}}}, {'name': 'dish_finder', 'type': 'integration', 'integration': {'provider': 'brave', 'method': 'search', 'setup': {'brave_api_key': 'BSAZmv1_2L3L8Dq38cGp9O338vDZ9nj'}}}, {'name': 'restaurant_finder', 'type': 'integration', 'integration': {'provider': 'brave', 'method': 'search', 'setup': {'brave_api_key': 'BSAZmv1_2L3L8Dq38cGp9O338vDZ9nj'}}}], 'main': [{'tool': 'weather_check', 'arguments': {'location': '$ steps[0].input.location'}, 'as': 'weather_data'}, {'tool': 'dish_finder', 'arguments': {'query': '$ "3 iconic local dishes of " + s

In [18]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

## Make Execution


In [23]:
def get_location_food_wise_details(location):
    execution = client.executions.create(
        task_id=task.id,
        input={'location':location}
    )

    while (result := client.executions.get(execution.id)).status not in ['succeeded', 'failed']:
        pass

    if result.status == "succeeded":
        print(result.output['choices'][0]['message']['content'])
        # print(type(result.output['choices']))
    else:
        print(f"Error: {result.error}")

In [29]:
cities=['mumbai','Abbeville','goa','Bali','New York City','London',"italy",'Paris']
for i in cities:
    get_location_food_wise_details(i)
    print("\n************************************************************************************************************************************"*4)
    

### Mumbai Food Adventure: 🎉 A Culinary Journey in the Heart of Hazy Mumbai

---

#### A Hazy Morning - The Culinary Prelude 🌥️🥭

Start your day with a sense of mystery as the hazy Mumbai morning wraps the city in a soft, gauzy embrace. The air is thick with humidity at 78%, lending a tropical feel that makes the aromas of sizzling street food even more enticing. With the current temperature at a delightfully warm 27°C (feeling like a cozy 29.5°C), it’s a perfect day to explore the bustling culinary landscape of Mumbai.

#### Breakfast at Anand Stall - A Vada Pav Ritual 🌶️🥖

Kickstart your morning at **Anand Stall** near Mithibai College—the locals’ haven for the best **Vada Pav** in town. Often referred to as Mumbai’s answer to the burger, this spicy potato fritter tucked between soft buns, with a slathering of tangy chutney, promises to awaken your senses. Each bite is a harmonious blend of textures and flavors—crispy, soft, spicy, and tangy—all at once. Vada Pav is best enjoyed with